In [19]:
import json 
import time 
import os 
from pathlib import Path 

from web3 import Web3
from etherscan import Etherscan
from dotenv import load_dotenv
import pandas as pd 

from flywheel_util.constants import url_infura, addresses
from flywheel_util.utils.util import ddf
from flywheel_util.utils.w3 import verified_contract

assert load_dotenv('../../.env')

precision = 2
pd.set_option('display.float_format', lambda x: f'%.{precision}f' % x)

In [20]:
w3 = Web3(Web3.HTTPProvider(url_infura))

hmmmm = 'VR7YA9DRDB4Y15B5N3WU9E7PSJ9RWPCP5S'
etherscan = Etherscan(hmmmm)

In [21]:
frax = verified_contract(addresses.token.frax)

In [22]:
# collateral pools pulled from the frax stablecoin smart contract. 
# https://dune.com/queries/1799531
collateral_pools = [
    '0x49ee75278820f409ecd67063d8d717b38d66bd71',
    '0x2fe065e6ffef9ac95ab39e5042744d695f560729',
    '0x3814307b86b54b1d8e7b2ac34662de9125f8f4e6',
    '0x96665d63c1b53f8335e3c9287ee255f306c93c45',
    '0x1864ca3d47aab98ee78d11fc9dcc5e7badda1c0d',
    '0xcf37b62109b537fa0cb9a90af4ca72f6fb85e241',
]

In [24]:
records = []
for a in collateral_pools: 
    contract, address_impl = verified_contract(a, return_address_impl=True)
    collat_usd = contract.functions.collatDollarBalance().call()
    code = etherscan.get_contract_source_code(address_impl) 
    assert len(code) == 1 
    name = code[0]['ContractName']
    r = {'name': name, 'address': a, 'address_impl': address_impl, 'collateral_usd': collat_usd / 1e18}
    records.append(r)
    time.sleep(.1)
df = pd.DataFrame(records)

In [25]:
print(df.collateral_usd.sum())
df.head(10)

1066364992.002416


,name,address,address_impl,collateral_usd
0,Convex_AMO,0x49ee75278820f409ecd67063d8d717b38d66bd71,0x49f77ddd4d57636ab4c98d8f18ca5f4b5210983d,385704122.93
1,FraxPoolV3,0x2fe065e6ffef9ac95ab39e5042744d695f560729,0x2fE065e6FFEf9ac95ab39E5042744d695F560729,10971153.08
2,UniV3LiquidityAMO,0x3814307b86b54b1d8e7b2ac34662de9125f8f4e6,0x3814307b86b54b1d8e7B2Ac34662De9125F8f4E6,11186812.00
3,RariFuseLendingAMO,0x96665d63c1b53f8335e3c9287ee255f306c93c45,0x96665d63c1B53f8335e3c9287Ee255f306C93c45,16152823.12
4,Pool_USDC,0x1864ca3d47aab98ee78d11fc9dcc5e7badda1c0d,0x1864Ca3d47AaB98Ee78D11fc9DCC5E7bADdA1c0d,500.05
5,FraxAMOMinter,0xcf37b62109b537fa0cb9a90af4ca72f6fb85e241,0xcf37B62109b537fa0Cb9A90Af4CA72f6fb85E241,642349580.83


In [27]:
frax_amo_minter_address = '0xcf37B62109b537fa0Cb9A90Af4CA72f6fb85E241'
contract = verified_contract(frax_amo_minter_address)
# collateral_addresses = contract.functions.allCollaterals().call()

In [29]:
amo_addrs = [a for a in contract.functions.allAMOAddresses().call() if a != '0x0000000000000000000000000000000000000000'] 
amo_addrs

['0x66635DC1EdEfF19e839f152a67278151Aa6e1B61',
 '0x6e1A844AFff1aa2a8ba3127dB83088e196187110',
 '0x8575FFE80b94fe58c8e0c735E11658A760109f53',
 '0xBA5478A712b5EA898AF03206ab4c7E0608C3e69D',
 '0xddf6b5B2BA110a0267BffB86AeAbFe2637cb8375',
 '0xc7F48Fb6Dbb6F8A3Eed90553017cDf5725Dc44ac',
 '0xF3a21b5d9E11eECA3a50BEb654276987164AbC8d',
 '0x4572B68296A23B4C66696FAd177c50CBc35c532F',
 '0xEc8672dd770D59FaD9a811591a5Edd40e8F6A413',
 '0x977eaDb6fa9b8E1a2A950CcDE1A75a7b527a8cBB',
 '0xc91Bb4b0696e3b48c0C501B4ce8E7244Fc363A79',
 '0x4828D4496Ff8dA2f5c1E7217Dc282b5C69B83263',
 '0x7F35dc487A5422D6946aAD733C6018F163084ed0',
 '0xa5924D9baA4fed0fbd100CB47CBCb61eA5E33219',
 '0x8D0A8f07F5Ea7F0982Ed116993200C331E4cdD27',
 '0xD4f638819Ed2c349E9Cfb6b5168d9e810a9D4f85',
 '0x629C473e0E698FD101496E5fbDA4bcB58DA78dC4',
 '0x0Ed8fA7FC63A8eb5487E7F87CAF1aB3914eA4eCa',
 '0xfC71e457Dea9bD129c05CF2036b11568b15e2217']

In [31]:
amo_collateral_holdings = []
for amo_addr in amo_addrs: 
    # AMO contract 
    c = verified_contract(amo_addr) 
    usd_bal = c.functions.dollarBalances().call()
    correction_offset = contract.functions.correction_offsets_amos(amo_addr, 1).call()
    code = etherscan.get_contract_source_code(amo_addr) 
    assert len(code) == 1 
    name = code[0]['ContractName']
    r = {'name': name, 'usd_balance': (usd_bal[1] + correction_offset) / 1e18, "address": amo_addr}
    print(r) 
    amo_collateral_holdings.append(r)
    time.sleep(.1)

{'name': 'AaveAMO', 'usd_balance': 1857704.5821125852, 'address': '0x66635DC1EdEfF19e839f152a67278151Aa6e1B61'}
{'name': 'FraxLiquidityBridger_POLY_MaticBridge', 'usd_balance': 20718974.020277362, 'address': '0x6e1A844AFff1aa2a8ba3127dB83088e196187110'}
{'name': 'FraxLiquidityBridger_FTM_AnySwap', 'usd_balance': 31895725.455375094, 'address': '0x8575FFE80b94fe58c8e0c735E11658A760109f53'}
{'name': 'FraxLiquidityBridger_AVAX_AnySwap', 'usd_balance': 8120998.311808184, 'address': '0xBA5478A712b5EA898AF03206ab4c7E0608C3e69D'}
{'name': 'FraxLiquidityBridger_ARBI_AnySwap', 'usd_balance': 231684.01379282467, 'address': '0xddf6b5B2BA110a0267BffB86AeAbFe2637cb8375'}
{'name': 'FraxLiquidityBridger_MOON_AnySwap', 'usd_balance': 12881170.0, 'address': '0xc7F48Fb6Dbb6F8A3Eed90553017cDf5725Dc44ac'}
{'name': 'FraxLiquidityBridger_SOL_WormholeV2', 'usd_balance': 1190.0, 'address': '0xF3a21b5d9E11eECA3a50BEb654276987164AbC8d'}
{'name': 'FraxLiquidityBridger_BSC_AnySwap', 'usd_balance': 0.0, 'address': 

In [32]:
df_amos = pd.DataFrame(amo_collateral_holdings)
df_amos = df_amos.sort_values('usd_balance', ascending=False).reset_index(drop=True) 
last_sync = df.loc[df.name == 'FraxAMOMinter'].collateral_usd.sum()
current = df_amos.usd_balance.sum() # more up to date than last sync so small discrepancies are expected 
print(f"Difference between actual and last sync collateral USDC held by FraxAMOMinter {(current - last_sync) / current:.2%}. Actual: ${current:,}")
ddf(df_amos)

Difference between actual and last sync collateral USDC held by FraxAMOMinter -0.21%. Actual: $640,992,968.1313468


,name,usd_balance,address
0,ManualTokenTrackerAMO,500985193.00,0xEc8672dd770D59FaD9a811591a5Edd40e8F6A413
1,FraxlendAMO,47133509.39,0x0Ed8fA7FC63A8eb5487E7F87CAF1aB3914eA4eCa
2,FraxLiquidityBridger_FTM_AnySwap,31895725.46,0x8575FFE80b94fe58c8e0c735E11658A760109f53
3,FraxLiquidityBridger_POLY_MaticBridge,20718974.02,0x6e1A844AFff1aa2a8ba3127dB83088e196187110
4,FraxLiquidityBridger_MOON_AnySwap,12881170.00,0xc7F48Fb6Dbb6F8A3Eed90553017cDf5725Dc44ac
5,FraxLiquidityBridger_AVAX_AnySwap,8120998.31,0xBA5478A712b5EA898AF03206ab4c7E0608C3e69D
6,FraxLiquidityBridger_BOBA_BobaGateway,6923601.95,0x4828D4496Ff8dA2f5c1E7217Dc282b5C69B83263
7,UniV3LiquidityAMO_V2,4803208.87,0xc91Bb4b0696e3b48c0C501B4ce8E7244Fc363A79
8,FraxLiquidityBridger_OPTI_Celer,2759740.73,0x7F35dc487A5422D6946aAD733C6018F163084ed0
9,TWAMM_AMO,2668000.00,0x629C473e0E698FD101496E5fbDA4bcB58DA78dC4
